In [ ]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob


In [55]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"]

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def read_txt_list():
  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[: , 1:]    
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]
    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:
           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "0"
        if data[i][3] > data[i][0]:
            sugg = "1"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [91]:
def download_data(symbols):
  
  indexx = 100
  work_with_dir()
  for symbol in symbols[:2]:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100

    try:
        data = yf.download(symbol, period="100d",interval="1h", progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 12:
             data.to_csv(f"/content/data/{symbol}.csv")
             
    except:
       print("Error!")
  print(" ")
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     data = np.array(unattached_dfs[0])
     for i in unattached_dfs[1:]:
           data = np.append(data, np.array(i), axis=0)
        
     unattached_dfs = []
  
     data = pd.DataFrame(data, columns=clmns)
     enc = OneHotEncoder(handle_unknown='ignore',sparse=False)
     cc = np.array(data["suggestion"]).reshape(-1,1)
     aa = enc.fit_transform(cc)
     #data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None

  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   #data = []
   return xTrain,xTest,yTrain,yTest,data
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [92]:
symbols = get_crypto_syms()
#symbols = read_txt_list()
#symbols = read_syms_from_txt()
#symbols = ["btc-usd","eth-usd","trx-usd"]

print(f"Symbols : {len(symbols)}")
download_data(symbols)
folder_name = extract_data(1)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest,data = make_df(folder_name)

Symbols : 1500
Data Folder Removed
 
Files In Data : 2
Processing File:
1 2  
(4754, 67)
(3803, 66) (3803,)
(951, 66) (951,)


In [93]:
data

,Open 12-11,Open 11-10,Open 10-9,Open 9-8,Open 8-7,Open 7-6,Open 6-5,Open 5-4,Open 4-3,Open 3-2,...,Volume 10-9,Volume 9-8,Volume 8-7,Volume 7-6,Volume 6-5,Volume 5-4,Volume 4-3,Volume 3-2,Volume 2-1,suggestion
0,-1.000000,-0.255839,-0.873708,0.461927,1.000000,-0.169786,0.085244,0.568226,-0.950663,-0.398673,...,-0.248926,-0.384839,-0.018081,-0.309874,-0.392344,-0.809034,-0.652997,0.284998,-0.392344,1
1,0.049212,-1.000000,-0.255839,-0.873708,0.461927,1.000000,-0.169786,0.085244,0.568226,-0.950663,...,-0.436427,0.103005,0.005390,0.268800,0.059231,0.000000,-0.299272,-0.187204,0.486476,0
2,0.271158,0.049212,-1.000000,-0.255839,-0.873708,0.461927,1.000000,-0.169786,0.085244,0.568226,...,0.440227,-0.594163,-0.205711,-0.276005,-0.086319,-0.237233,-0.279886,-0.495396,-0.414695,1
3,-0.203481,0.271158,0.049212,-1.000000,-0.255839,-0.873708,0.461927,1.000000,-0.169786,0.085244,...,-1.000000,0.440227,-0.594163,-0.205711,-0.276005,-0.086319,-0.237233,-0.279886,-0.495396,0
4,1.000000,-0.481641,-0.172755,-0.317194,-1.000000,-0.515715,-0.917811,-0.048606,0.301561,-0.459713,...,1.000000,-0.967773,0.449247,-0.568475,-0.186283,-0.255444,-0.068815,-0.217297,-0.259263,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4749,-0.166149,0.254640,0.349280,1.000000,0.469135,0.695607,-1.000000,0.348024,0.234390,0.690003,...,-0.101446,-0.093445,-0.543986,-1.000000,1.000000,0.198198,-0.159436,-0.047859,-0.094904,0
4750,0.217264,-0.166149,0.254640,0.349280,1.000000,0.469135,0.695607,-1.000000,0.348024,0.234390,...,0.036423,-0.101446,-0.093445,-0.543986,-1.000000,1.000000,0.198198,-0.159436,-0.047859,1
4751,0.529905,0.217264,-0.166149,0.254640,0.349280,1.000000,0.469135,0.695607,-1.000000,0.348024,...,0.069098,0.036423,-0.101446,-0.093445,-0.543986,-1.000000,1.000000,0.198198,-0.159436,0
4752,1.000000,0.453488,0.156463,-0.207799,0.191972,0.281885,0.900102,0.395754,0.610913,-1.000000,...,-0.416829,0.069098,0.036423,-0.101446,-0.093445,-0.543986,-1.000000,1.000000,0.198198,0


In [105]:
model = Sequential()



model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))


opt = tf.keras.optimizers.Adamax()

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_59 (Dense)            (None, 1024)              68608     
                                                                 
 dense_60 (Dense)            (None, 512)               524800    
                                                                 
 dense_61 (Dense)            (None, 512)               262656    
                                                                 
 dense_62 (Dense)            (None, 1)                 513       
                                                                 
Total params: 856,577
Trainable params: 856,577
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=100,batch_size=1000,validation_data=(xTest,yTest),shuffle=False)

Epoch 1/100
4/4 [==============================] - 1s 185ms/step - loss: 0.7035 - accuracy: 0.5064 - val_loss: 0.6972 - val_accuracy: 0.5279
Epoch 2/100
4/4 [==============================] - 0s 126ms/step - loss: 0.6741 - accuracy: 0.5809 - val_loss: 0.6917 - val_accuracy: 0.5300
Epoch 3/100
4/4 [==============================] - 0s 124ms/step - loss: 0.6633 - accuracy: 0.6208 - val_loss: 0.6912 - val_accuracy: 0.5478
Epoch 4/100
4/4 [==============================] - 0s 123ms/step - loss: 0.6517 - accuracy: 0.6450 - val_loss: 0.6901 - val_accuracy: 0.5605
Epoch 5/100
4/4 [==============================] - 0s 126ms/step - loss: 0.6368 - accuracy: 0.6739 - val_loss: 0.6873 - val_accuracy: 0.5584
Epoch 6/100
4/4 [==============================] - 0s 122ms/step - loss: 0.6234 - accuracy: 0.6821 - val_loss: 0.6860 - val_accuracy: 0.5636
Epoch 7/100
4/4 [==============================] - 0s 124ms/step - loss: 0.6061 - accuracy: 0.7065 - val_loss: 0.6851 - val_accuracy: 0.5594
Epoch 8/100
4

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
